In [94]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["ZHIPUAI_API_KEY"] = "8b4fa95036a9d5a9f7a9ad2156b36e5f.ePH8ShMykmW3Obqk"

In [95]:
from langchain_community.chat_models import ChatZhipuAI

model = ChatZhipuAI(name='glm-4-flash')

In [96]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory

# 存储所有对话内容
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [97]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [98]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([("system", "你是一个有关{species}的农业问答助手，"
                                                      "请你利用所学知识生成准确科学的答案。"),
                                           MessagesPlaceholder(variable_name="messages"), ])

chain = prompt | model

In [99]:
from langchain_core.messages import trim_messages

trimmer = trim_messages(max_tokens=2000,
                        strategy="last",
                        token_counter=model,
                        include_system=True,
                        allow_partial=False,
                        start_on="human")

In [100]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

trimmer = RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)

chain = trimmer | prompt | model

In [101]:
from langchain_core.runnables.history import RunnableWithMessageHistory

bot = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [102]:
config = {"configurable": {"session_id": "abc1"}}

In [103]:
input = {"messages": [HumanMessage(content="你好！")],
         "species": "核桃"}

In [105]:
for output in bot.stream(input, config=config):
    print(output.content, end='')

你好！很高兴能帮助你解答关于核桃的任何问题。如果你有任何疑问或需要咨询的信息，请随时告诉我。